# PID Control - Laboratory

## PID Control

The following Python code creates a PID generator that can be use for feedback control.  This the same generator we developed in class on Thursday, with an additional filter used for a more robust implementation of derivative action.  In today's laboratory you will use this code to test several aspects of feedback control of the TCLab heater.

In [ ]:
def PID(Kp, Ki, Kd, MV_bar=0, beta=1, gamma=0):
    # initialize stored data
    t_prev = -100
    P = 0
    I = 0
    D = 0
    S = 0
    N = 10
    
    # initial control
    MV = MV_bar
    
    while True:
        # yield MV, wait for new t, SP, PV, TR
        data = yield MV
        
        # see if a tracking data is being supplied
        if len(data) < 4:
            t, SP, PV = data
        else:
            t, SP, PV, TR = data
            I = TR - MV_bar - P - D
        
        # PID calculations
        P = Kp*(beta*SP - PV)
        I = I + Ki*(SP - PV)*(t - t_prev)
        eD = gamma*SP - PV
        D = N*Kp*(Kd*eD - S)/(Kd + N*Kp*(t - t_prev))
        MV = MV_bar + P + I + D
        
        # Constrain MV to range 0 to 100 for anti-reset windup
        MV = 0 if MV < 0 else 100 if MV > 100 else MV
        I = MV - MV_bar - P - D
        
        # update stored data for next iteration
        S = D*(t - t_prev) + S
        t_prev = t

## Exercise 1.  Is the transfer from manual to automatic bumpless?

The cell below is a starter code to implement feedback control. The code establishes a setpoint of 40°C, and for the first 30 seconds runs under manual control. The control then transfers to automatic. Do you observe bumpless transfer?  Explain what you see.

In [ ]:
%matplotlib inline
from tclab import clock, setup, Historian, Plotter

TCLab = setup(connected=True)

controller = PID(Kp=2, Ki=0.1, Kd=2, beta=0)   # create pid control
controller.send(None)                          # initialize

tfinal = 600
SP = 40

with TCLab() as lab:
    sources = [('SP', lambda: SP), ('T1', lambda: lab.T1), ('MV', lambda: MV), ('Q1', lab.Q1)]
    h = Historian(sources)
    p = Plotter(h, tfinal)
    for t in clock(tfinal, 2):
        PV = lab.T1                            # get measurement
        MV = controller.send([t, SP, PV, lab.U1])  # for bumpless transfer
        if t < 100:                            # choice between manual and auto control
            MV = 50
        lab.U1 = MV                            # apply 
        p.update(t)                            # update information display

In [2]:
from tclab import diagnose

diagnose()

Looking for Arduino on  ...
NHduino found on port /dev/cu.wchusbserial1410

Testing TCLab object in debug mode
----------------------------------
TCLab version 0.4.6dev
Sent: "Q1 0"
Return: ""
Sent: "Q1 0"
Return: "0"
Could not connect at high speed, but succeeded at low speed.
This may be due to an old TCLab firmware.
New Arduino TCLab firmware available at:
 https://github.com/jckantor/TCLab-sketch
Sent: "VER"
Return: "TCLab Firmware Version 1.2.1"
NHduino connected on port /dev/cu.wchusbserial1410 at 9600 baud.
TCLab Firmware Version 1.2.1.
Sent: "Q2 0"
Return: "0"
Reading temperature
Sent: "T1"
Return: "27.02"
27.02
Sent: "Q1 0"
Return: "0"
Sent: "Q2 0"
Return: "0"
Sent: "X"
Return: "X"
TCLab disconnected successfully.

Testing TCLab functions
-----------------------
TCLab version 0.4.6dev
Could not connect at high speed, but succeeded at low speed.
This may be due to an old TCLab firmware.
New Arduino TCLab firmware available at:
 https://github.com/jckantor/TCLab-sketch
NHduino c

## Exercise 2

Create a new experiment in the cell below.  The new experiment should do the following:

* Add a second PID controller for the second heater.  The control constants should be Kp=4, Ki=0.2, Kd=2, beta=0.
* Should start up under automatic control with setpoints of 40 deg C for both heaters.  
* At t = 100, the setpoint for heater 1 should be raised to 50 deg C
* You will need to modify the 'sources' line to include the additional data.  Please see an instructor if you need help with this.

Run the experiment for at least 800 seconds.  How would describe the behavior at the point when the setpoint is changed?

In [ ]:
from tclab import diagnose
diagnose()


Looking for Arduino on  ...
NHduino found on port /dev/cu.wchusbserial1410

Testing TCLab object in debug mode
----------------------------------
TCLab version 0.4.6dev
Sent: "Q1 0"
Return: ""
Sent: "Q1 0"
Return: "0"
Could not connect at high speed, but succeeded at low speed.
This may be due to an old TCLab firmware.
New Arduino TCLab firmware available at:
 https://github.com/jckantor/TCLab-sketch
Sent: "VER"
Return: "TCLab Firmware Version 1.2.1"
NHduino connected on port /dev/cu.wchusbserial1410 at 9600 baud.
TCLab Firmware Version 1.2.1.
Sent: "Q2 0"
Return: "0"
Reading temperature
Sent: "T1"
Return: "21.54"
21.54
Sent: "Q1 0"
Return: "0"
Sent: "Q2 0"
Return: "0"
Sent: "X"
Return: "X"
TCLab disconnected successfully.

Testing TCLab functions
-----------------------
TCLab version 0.4.6dev
Could not connect at high speed, but succeeded at low speed.
This may be due to an old TCLab firmware.
New Arduino TCLab firmware available at:
 https://github.com/jckantor/TCLab-sketch
NHduino c